# Task 3: Model Explainability using SHAP and LIME

In [45]:
# Import necessary libraries
import shap
import lime
from lime.lime_tabular import LimeTabularExplainer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import joblib
from sklearn.model_selection import train_test_split

In [46]:
# Define directory for saved models
model_dir = r"C:/Users/Administrator/Documents/kifiya/Week_8/saved_models"

In [48]:
# Load PyTorch models
def load_pytorch_model(model_name):
    model_path = f"{model_dir}/{model_name}.pt"
    model = torch.load(model_path)
    model.eval()  # Set model to evaluation mode
    
    return model

# Load scikit-learn models
def load_sklearn_model(model_name):
    model_path = f"{model_dir}/{model_name}.joblib"
    model = joblib.load(model_path)
    return model


In [ ]:
input_size = 30
# Dictionary of loaded models for credit card and fraud datasets
models_credit = {
    'MLP_Credit': load_pytorch_model('MLP_Credit'),
    'CNN_Credit': load_pytorch_model('CNN_Credit'),
    'RNN_Credit': load_pytorch_model('RNN_Credit'),
    'LSTM_Credit': load_pytorch_model('LSTM_Credit'),
    'DecisionTree_Credit': load_sklearn_model('DecisionTree_Credit'),
    'GradientBoosting_Credit': load_sklearn_model('GradientBoosting_Credit'),
    'LogisticRegression_Credit': load_sklearn_model('LogisticRegression_Credit'),
    'RandomForest_Credit': load_sklearn_model('RandomForest_Credit')
}

models_fraud = {
    'MLP_Fraud': load_pytorch_model('MLP_Fraud'),
    'CNN_Fraud': load_pytorch_model('CNN_Fraud'),
    'RNN_Fraud': load_pytorch_model('RNN_Fraud'),
    'LSTM_Fraud': load_pytorch_model('LSTM_Fraud'),
    'DecisionTree_Fraud': load_sklearn_model('DecisionTree_Fraud'),
    'GradientBoosting_Fraud': load_sklearn_model('GradientBoosting_Fraud'),
    'LogisticRegression_Fraud': load_sklearn_model('LogisticRegression_Fraud'),
    'RandomForest_Fraud': load_sklearn_model('RandomForest_Fraud')
}


In [24]:
# Load processed datasets if needed
fraud_data = pd.read_csv('C:/Users/Administrator/Documents/kifiya/Week_8/clean_data/merged_data.csv')
creditcard_data = pd.read_csv('C:/Users/Administrator/Documents/kifiya/Week_8/clean_data/Preprocessed_Creditcard_Data.csv')

# dropping uneccessary rows
fraud_data = fraud_data.drop(columns=['signup_time', 'purchase_time', 'user_id', 'device_id', 
                                      'ip_address', 'lower_bound_ip_address', 'upper_bound_ip_address'], errors='ignore')

print('The Merged fraud data')
display(fraud_data.head())
print('credit data')
display(creditcard_data.head())

The Merged fraud data


,purchase_value,sex,age,class,transaction_count,hour_of_day,day_of_week,purchase_value_scaled,source_Direct,source_SEO,...,country_United States,country_Uruguay,country_Uzbekistan,country_Vanuatu,country_Venezuela,country_Viet Nam,country_Virgin Islands (U.S.),country_Yemen,country_Zambia,country_Zimbabwe
0,47.0,0,30.0,0.0,1,3,6,0.549607,False,True,...,False,False,False,False,False,False,False,False,False,False
1,15.0,0,34.0,0.0,1,20,2,-1.197335,False,True,...,False,False,False,False,False,False,False,False,False,False
2,44.0,1,29.0,0.0,1,23,5,0.385831,False,False,...,False,False,False,False,False,False,False,False,False,False
3,55.0,0,30.0,0.0,1,16,5,0.986342,True,False,...,False,False,False,False,False,False,False,False,False,False
4,51.0,0,37.0,0.0,1,4,1,0.767974,False,True,...,False,False,False,False,False,False,False,False,False,False


credit data


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,time_in_days,Amount_scaled
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,0.000000,0.244200
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,0.000000,-0.342584
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,0.000012,1.158900
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,0.000012,0.139886
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,0.000023,-0.073813


In [25]:
# Separate features and target for both datasets
X_credit = creditcard_data.drop(columns=['Class']).values
y_credit = creditcard_data['Class'].values

X_fraud = fraud_data.drop(columns=['class']).values
y_fraud = fraud_data['class'].values

# Split data into train and test sets for both datasets
X_train_credit, X_test_credit, y_train_credit, y_test_credit = train_test_split(X_credit, y_credit, test_size=0.3, random_state=42)
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.3, random_state=42)

In [ ]:
### SHAP Analysis for scikit-learn models ###
# SHAP explainers and plots for scikit-learn models (Credit and Fraud)
for dataset_name, X_test, models in [('Credit', X_test_credit, models_credit), ('Fraud', X_test_fraud, models_fraud)]:
    for model_name, model in models.items():
        if isinstance(model, (joblib.tree.DecisionTreeClassifier, joblib.ensemble.RandomForestClassifier)):
            explainer = shap.TreeExplainer(model)
        else:
            explainer = shap.KernelExplainer(model.predict, X_test)
        
        shap_values = explainer.shap_values(X_test)

        # SHAP Summary Plot
        plt.title(f'SHAP Summary Plot for {model_name}_{dataset_name}')
        shap.summary_plot(shap_values, X_test, plot_type="bar")
        plt.show()

        # SHAP Force Plot for the first instance
        plt.title(f'SHAP Force Plot for {model_name}_{dataset_name} - First Instance')
        shap.force_plot(explainer.expected_value[1], shap_values[1][0], X_test[0], matplotlib=True)
        plt.show()

In [ ]:
### SHAP Analysis for PyTorch models (deep learning models) ###
# SHAP for deep learning models (Credit and Fraud)
for dataset_name, X_test, X_train, models in [('Credit', X_test_credit, X_train_credit, models_credit), ('Fraud', X_test_fraud, X_train_fraud, models_fraud)]:
    for model_name, model in models.items():
        if model_name.startswith(('MLP', 'CNN', 'RNN', 'LSTM')):
            explainer = shap.DeepExplainer(model, torch.tensor(X_train).float())
            shap_values = explainer.shap_values(torch.tensor(X_test).float())

            # SHAP Summary Plot
            plt.title(f'SHAP Summary Plot for {model_name}_{dataset_name}')
            shap.summary_plot(shap_values, X_test, plot_type="bar")
            plt.show()

In [ ]:
### LIME Analysis for scikit-learn models ###
# LIME for both datasets
for dataset_name, X_test, feature_names, models in [('Credit', X_test_credit, creditcard_data.columns[:-1], models_credit), 
                                                    ('Fraud', X_test_fraud, fraud_data.columns[:-1], models_fraud)]:
    lime_explainer = LimeTabularExplainer(
        X_test,
        feature_names=feature_names,
        class_names=['Non-Fraud', 'Fraud'],
        mode='classification'
    )
    
    # Generate LIME explanation for a single instance (e.g., the first fraud prediction)
    instance_idx = 0  # Adjust for specific fraud cases as needed
    for model_name, model in models.items():
        if not model_name.startswith(('MLP', 'CNN', 'RNN', 'LSTM')):  # Only for scikit-learn models
            exp = lime_explainer.explain_instance(X_test[instance_idx], model.predict_proba)

            # Display LIME explanation in the notebook
            print(f"LIME Explanation for instance {instance_idx} - Model: {model_name}_{dataset_name}")
            exp.show_in_notebook(show_table=True)
            exp.as_pyplot_figure()
            plt.title(f'LIME Feature Importance for Single Prediction - {model_name}_{dataset_name}')
            plt.show()
